In [7]:
import tensorflow as tf
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
from function import question_processing,tokenize_and_filter,tokenizer
keras = tf.keras
t = Okt()

In [8]:
tempmodel = keras.models.load_model('entitytestmodel.h5')
tempmodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          5504      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 195       
Total params: 80,003
Trainable params: 80,003
Non-trainable params: 0
_________________________________________________________________


In [9]:
intentdf = pd.read_csv('train_intent.csv',encoding="CP949")
entitydf = pd.read_csv('Entitytest.csv',encoding = "CP949")
print(entitydf[0:5])
intenttoken = tokenizer()
intenttoken.fit(intentdf['question'].values)

entitytoken = tokenizer()
entitytoken.fit(entitydf['word'].values)

intentmodel = keras.models.load_model('my_model.h5')
entitymodel = keras.models.load_model('entitytestmodel.h5')

def 형태소분석(text):
    형태소 = kkma.pos(text)
    명사 = []
    for i in 형태소:
        if i[1] == 'NNG':
            명사.append(i[0])
        else:
            pass
    return 명사

def entity분석(명사):
    inputdata = question_processing(명사,entitytoken)
    prediction = list(np.argmax(entitymodel.predict(inputdata),axis=1))
    print(명사)
    print(prediction)
    result = {}
    for a,b in zip(명사,prediction):
        if b == 0:
            result[a] = 'color'
        elif b == 1:
            result[a] = 'thing'
        elif b == 2:
            result[a] = 'loc'
    return result  

  word entity
0  빨간색  color
1  파랑색  color
2  빨강색  color
3  파란색  color
4  주황색  color
scanning was done                                        
64 terms are recognized
scanning was done                                        
86 terms are recognized


In [11]:
input_sentence = question_processing(['빨간색','휴지','파랑색','비','무지개색','오른쪽','노란색','유니폼','상자','옆','위'
                                     ],entitytoken)

In [12]:
tempmodel.predict(input_sentence)

array([[9.4229478e-01, 3.1282444e-04, 5.7392407e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.4701344e-01, 2.4948525e-04, 5.2737065e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [9.6862912e-01, 3.1206716e-04, 3.1058801e-02],
       [4.8841119e-02, 8.9883851e-03, 9.4217056e-01],
       [9.4959462e-01, 3.2338305e-04, 5.0081894e-02],
       [9.3867078e-05, 9.9887842e-01, 1.0277359e-03],
       [4.1759838e-05, 9.9968314e-01, 2.7513146e-04],
       [3.4237865e-02, 1.2231455e-02, 9.5353067e-01],
       [2.2944862e-02, 1.5088378e-02, 9.6196675e-01]], dtype=float32)

In [14]:
prediction = np.argmax(tempmodel.predict(input_sentence), axis=1)
print(prediction)

[0 1 0 1 0 2 0 1 1 2 2]


In [1]:
import tensorflow as tf
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt


keras = tf.keras
t = Okt()

In [2]:
from vectorizer import BaseVectorizer
vectorizer = BaseVectorizer(t.morphs)
tokenizer = BaseVectorizer(t.morphs)

In [3]:
df = pd.read_csv('Entitytest.csv',encoding='CP949')

In [4]:
tokenizer.fit(df['word'].values)

scanning was done                                        
105 terms are recognized


In [5]:
tokenizer.get_char2idx()

In [6]:
wordvoca = tokenizer.char2idx

In [7]:
tempmodel = keras.models.load_model('entitytestmodel.h5')
tempmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          8320      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 82,819
Trainable params: 82,819
Non-trainable params: 0
_________________________________________________________________


In [8]:
def word_question_procession(words):
    MAX_LENGTH = 10
    inputs = [] 
    for word in words:
        tempword = []
        tempnum = []
        for i in range(len(word)):
            tempword.append(word[i])
        for i in tempword:
            try:
                tempnum.append(wordvoca[i])
            except:
                pass
        if len(tempnum) <= MAX_LENGTH:
            inputs.append(tempnum)
        else:
            print("단어의 길이가 너무 길어요")
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.char2idx['_PAD_']) # value = 0
    return padded_inputs

In [9]:
input_sentence = word_question_procession(['빨간색','휴지','파랑색','비','무지개색','오른쪽','노란색','유니폼','상자','옆','위','뿡뿡색'
                                     ])

In [10]:
tempmodel.predict(input_sentence)

array([[9.99997258e-01, 2.36760138e-06, 3.79820023e-07],
       [4.01552606e-05, 9.99946475e-01, 1.33432895e-05],
       [9.99997973e-01, 1.85279464e-06, 1.44728048e-07],
       [1.45368362e-02, 9.83871937e-01, 1.59128278e-03],
       [7.20760524e-01, 2.78951228e-01, 2.88266368e-04],
       [6.82854501e-04, 3.85420630e-04, 9.98931706e-01],
       [9.99993563e-01, 6.26468636e-06, 1.59969190e-07],
       [2.25275682e-04, 9.99717891e-01, 5.68391297e-05],
       [7.61514675e-05, 9.99898911e-01, 2.49588320e-05],
       [4.26918385e-04, 3.24093970e-04, 9.99248922e-01],
       [4.10627166e-04, 3.02611763e-04, 9.99286830e-01],
       [9.87252474e-01, 9.90190543e-03, 2.84563052e-03]], dtype=float32)

In [14]:
prediction = list(np.argmax(tempmodel.predict(input_sentence), axis=1))

In [16]:
prediction

[0, 1, 0, 1, 0, 2, 0, 1, 1, 2, 2, 0]